# Decorator by a Function

[Decorator (function) utilize the concept of "closure" in Python](https://www.youtube.com/watch?v=FsAPt_9Bf3U)

# Decorator for a Function Wrapper

Usage: adding extra functionalities to a function without modifying codes for calling of the function.

Normally, we add extra functionalities to a function by changing the function itself. However, there are cases we want to keep the original function unchanged, and "wrap" functionalities around the original function (for example, logging information of a function like execution time, input, output, etc).

The general form of a decorator is:

In [111]:
def decorator_function(original_function): # specify the name of the original function to be decorated

    def wrapper(*args, **kwargs): # details for the new function
        print('You can add functionalities inside a wrapper here')
        result_of_original_function = original_function(*args, **kwargs)
        print('Just to remember to return the original result')
        return result_of_original_function
    
    return wrapper # specify the name of the new function to substitute the original function

@decorator_function # specify the decorator
def say_something(message):
    return 'I want to say ' + message

print(say_something('hi'))

You can add functionalities inside a wrapper here
Just to remember to return the original result
I want to say hi


### Why not just assign a new name instead of using a decorator

At first sight, one might think the idea of decorator is redundant because the same effect can be achieved by the following (`@decorator_function` removed and assign the function `say_something` to `decorator_function(say_something)`:

In [112]:
def decorator_function(original_function):

    def wrapper(*arg, **kargs):
        print('You can add functionalities inside a wrapper here')
        result_of_original_function = original_function(*arg, **kargs)
        print('Just to remember to return the original result')
        return result_of_original_function

    return wrapper

def say_something(message):
    return 'I want to say ' + message

print('Call with wrapper:')
say_something = decorator_function(say_something)
print(say_something('hi'))

Call with wrapper:
You can add functionalities inside a wrapper here
Just to remember to return the original result
I want to say hi


Indeed, `say_something = decorator_function(say_something)` is actually what was done by `@decorator_function`. However, the above approach is not a good idea. Because one may eventually forgot what the function `say_something` actually mean when the code grows. The situation exacerbates when we have many different decorations (say, one for loggin and another for timing) to the same function.

One the other hand, one might say we could just call the function `wrapper` instead of `say_something`. But this requires change to EVERY call in pre-existed codes, which is what we want to avoid at the first place.

Note that, the same decorator can be applied to different functions if the decorator is written in a general form. This further strengthen the reason to use decorators (one log decorator or one timing decorator for any function) instead of name re-assignment.

### Example code

Note that the default key word argument is not recorded.

In [113]:
def log_input(target_function):
    def wrapper(*args, **kwargs):
        input_variables = [arg for arg in args] + [kwarg for kwarg in kwargs]
        print('The input variables are:')
        print(input_variables)
        result = target_function(*args)
        return result
    return wrapper

def log_output(target_function):
    def wrapper(*args, **kwargs):
        result = target_function(*args)
        print('The output is:', result)
        return result
    return wrapper


# @log_output
@log_input
def summation(*args, absolute=True):
    total = 0
    for arg in args:
        total += arg
    if absolute:
        total = abs(total)
    return total


a = (1, 2, 3, 4, 5)

summation(*a)

The input variables are:
[1, 2, 3, 4, 5]


15

### Stacking decorators

It's possible to use multiple decoration on the same function. The stacking

```Python
@third
@second
@first
def original_function():
```

is equivalent to 

```python
original_function = third(second(first(original_function)))
```

In [114]:
@log_output # executed second
@log_input # executed first
def summation(*args, absolute=True):
    total = 0
    for arg in args:
        total += arg
    if absolute:
        total = abs(total)
    return total

a = (1, 2, 3, 4, 5)
summation(*a, True)

The input variables are:
[1, 2, 3, 4, 5, True]
The output is: 16


16

### Function passed in stacked decorators

One should be careful about stacking the decorators. The variabls passed to another decorator might be confusing. Consider the following stacked decoration:

In [121]:
def decorator(funct):
    def wrapper(*args):
        print('The name of the decorated function:', funct.__name__)
        result = funct(*args)
        return result
        # return funct(*args) # ??what happen in this case??
    return wrapper

@decorator
@decorator
def message(text):
    print(text)

message('Hello')

The name of the decorated function: wrapper
The name of the decorated function: message
Hello


The two decoration is equivalent to `decorator(decorator(message))`.

- The first decoration, namely `decorator(message)`, has the input function `message` and returns the function `wrapper`. 
- The second decoration, namely `decorator(decorator(message))`, is equivalent to `decorator(wrapper)`, where `wrapper` is the returned function from the first decoration, so the input function is now `wrapper`.
- Python first see the outer function (second decoration) and then the inner function (first decoration), so it prints `wrapper` function name and then `message` function name.

### Preserve function name in stacked decorators

It would be great if we can obtain the original function instead of wrapper in decoration. This can be achieved by using the `wraps` decorator in the module `functools` and decorate all wrapper functions:

In [122]:
from functools import wraps

def decorator(funct):
    @wraps(funct)
    def wrapper(*args):
        print('The name of the decorated function:', funct.__name__)
        result = funct(*args)
        return result
        # return funct(*args) # ??what happen in this case??
    return wrapper

@decorator
@decorator
def message(text):
    print(text)

message('Hello')

The name of the decorated function: message
The name of the decorated function: message
Hello


### ?? Confusion in stacked decorator??

why lvl are different in the two calls?

In [116]:
lvl = 1

def decorator1(base):
    def wrapper(a):
        global lvl
        print('input:', a, 'lvl:', lvl)
        result = base(a*2)
        lvl += 1
        return result
    return wrapper

def decorator2(base):
    def wrapper(a):
        global lvl
        print('input:', a, 'lvl:', lvl)
        lvl += 1
        return base(a*2)
    return wrapper

@decorator1
@decorator1
@decorator1
def base(a):
    return a

base(2)

@decorator2
@decorator2
@decorator2
def base(a):
    return a

base(2)


input: 2 lvl: 1
input: 4 lvl: 1
input: 8 lvl: 1
input: 2 lvl: 4
input: 4 lvl: 5
input: 8 lvl: 6


16

# Decorator by a class

Similar decoration effect can also be achieved by using a class. It has more utilities but less used.

In [117]:
class decorator_class:

    def __init__(self, original_function):
        self.original_function = original_function

    def __call__(self, *args, **kwargs): # serves like wrapper
        print('You can add functionalities inside a call here')
        result_of_original_function = self.original_function(*args, **kwargs)
        print('Just to remember to return the original result')
        return result_of_original_function

@decorator_class # specify the decorator
def say_something(message):
    return 'I want to say ' + message

print(say_something('hi'))

You can add functionalities inside a call here
Just to remember to return the original result
I want to say hi


# Reference

[dyanmic functionality; stacked decorators;wrapper using class](https://www.youtube.com/watch?v=FsAPt_9Bf3U)

# Future Studies

The variable scoping rule in class may look different from typical python functions as mentioned [in the SECOND answer](https://stackoverflow.com/questions/51117397/why-method-cant-access-class-variable) on stackoverflow.

[the super() magic of class inheritance](https://stackoverflow.com/questions/19608134/why-is-python-3-xs-super-magic)

[iterable and iterator](https://www.w3schools.com/python/python_iterators.asp)

[name alias; first-class object](https://stackoverflow.com/questions/28309757/instancing-a-class-difference-between-with-and-without-brackets)

[order of "not", "and", and "or" operations and non-boolean inputs](https://en.wikibooks.org/wiki/Python_Programming/Operators#Logical_Operators)